In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt

In [57]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [16]:
all_train_snp = pd.read_csv("./csv/all_train_snp_selected.csv")
validation_snp = pd.read_csv("./csv/validation_snp_selected.csv")

In [23]:
X_train_ = all_train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
X_val_ = validation_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
y_train = all_train_snp["target"] - 1
y_val = validation_snp["target"] - 1

In [24]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train_)
X_val = ss.transform(X_val_)

model_gb = LogisticRegression()
model_gb.fit(X_train, y_train)

y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC autoencoded: {roc_auc:.4f}')

ROC-AUC autoencoded: 0.6361


In [95]:
rocs = []
for i in range(1, 6):
    train_snp = pd.read_csv(f"./csv/train_{i}_snp_selected.csv")
    test_snp = pd.read_csv(f"./csv/test_{i}_snp_selected.csv")

    X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
    y_train = train_snp["target"] - 1
    y_val = test_snp["target"] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_)
    X_val = ss.transform(X_val_)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_val, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC autoencoded: {roc_auc:.4f}')
print("среднее", np.mean(rocs))

ROC-AUC autoencoded: 0.5603
ROC-AUC autoencoded: 0.5720
ROC-AUC autoencoded: 0.5833
ROC-AUC autoencoded: 0.5145
ROC-AUC autoencoded: 0.6296
среднее 0.5719375440073178


In [30]:
input_dim = X_train_.shape[1]
encoding_dim = 16
batch_size = 128
num_epochs = 90
learning_rate = 0.001

X_train_tensor = torch.FloatTensor(X_train_)
X_val_tensor = torch.FloatTensor(X_val_)

train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

models = []
for i in range(1):
    model = Autoencoder(input_dim, encoding_dim)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch_inputs, batch_targets in train_loader:
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * batch_inputs.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_inputs, batch_targets in val_loader:
                outputs = model(batch_inputs)
                loss = criterion(outputs, batch_targets)
                val_loss += loss.item() * batch_inputs.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)
    models.append(model)

trains = []
vals = []
for model in models:
    encoder = model.encoder
    train_encoded = encoder(torch.FloatTensor(X_train_))
    trains.append(train_encoded)
    val_encoded = encoder(torch.FloatTensor(X_val_))
    vals.append(val_encoded)
train_encoded = torch.cat(trains, axis=1)
val_encoded = torch.cat(vals, axis=1)

with torch.no_grad():
    selector = SelectKBest(f_classif, k=10)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    ss = StandardScaler()
    with torch.no_grad():
        X_train = ss.fit_transform(X_train_s)
        X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_rr = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_rr:.4f}')

with torch.no_grad():
    selector = SelectKBest(f_classif, k=40)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
    X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)

    ss = StandardScaler()
    with torch.no_grad():
        X_train = ss.fit_transform(X_train_s)
        X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_main = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_main:.4f}')

ROC-AUC train autoencoded: 0.6121
ROC-AUC autoencoded: 0.5924
ROC-AUC train autoencoded: 0.7164
ROC-AUC autoencoded: 0.6439


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 2  5  8 11] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=40 is greater than n_features=16. All the features will be returned.
  warnings.warn(
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 2  5  8 11] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selec

In [43]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def objective(trial):
    encoding_dim = trial.suggest_int('encoding_dim', 16, 64)
    batch_size = 128
    num_epochs = 65
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    
    hidden_dim = 186
    
    k_select = 75
    
    lr_C = trial.suggest_float('lr_C', 1e-3, 10, log=True)
    
    input_dim = X_train_.shape[1]
    
    X_train_tensor = torch.FloatTensor(X_train_)
    X_val_tensor = torch.FloatTensor(X_val_)
    
    train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    class Autoencoder(nn.Module):
        def __init__(self, input_dim, encoding_dim, hidden_dim):
            super(Autoencoder, self).__init__()
            
            self.encoder = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, encoding_dim),
                nn.ReLU()
            )
            
            self.decoder = nn.Sequential(
                nn.Linear(encoding_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, input_dim)
            )
            
        def forward(self, x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return decoded
    
    models = []
    model = Autoencoder(input_dim, encoding_dim, hidden_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch_inputs, batch_targets in train_loader:
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * batch_inputs.size(0)
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_inputs, batch_targets in val_loader:
                outputs = model(batch_inputs)
                loss = criterion(outputs, batch_targets)
                val_loss += loss.item() * batch_inputs.size(0)
    
    models.append(model)
    
    trains = []
    vals = []
    for model in models:
        encoder = model.encoder
        train_encoded = encoder(torch.FloatTensor(X_train_))
        trains.append(train_encoded)
        val_encoded = encoder(torch.FloatTensor(X_val_))
        vals.append(val_encoded)
    train_encoded = torch.cat(trains, axis=1)
    val_encoded = torch.cat(vals, axis=1)
    
    with torch.no_grad():
        selector = SelectKBest(f_classif, k=k_select)
        X_train_s = selector.fit_transform(train_encoded, y_train)
        X_val_s = selector.transform(val_encoded)
        
        X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
        X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)
        
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train_s)
        X_val = ss.transform(X_val_s)
        
        model_gb = LogisticRegression(C=lr_C, max_iter=1000)
        model_gb.fit(X_train, y_train)
        
        y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
        roc_auc_main = roc_auc_score(y_val, y_pred_proba)
    
    return roc_auc_main

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")
print(f"Лучшее значение ROC-AUC: {study.best_value:.4f}")


[I 2025-04-24 04:35:36,222] A new study created in memory with name: no-name-b3ab1218-c9cf-4c1c-8bbe-e13be7a77b58
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=75 is greater than n_features=17. All the features will be returned.
  warnings.warn(
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [10 16] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
[I 2025-04-24 04:35:38,735] Trial 0 finished with value: 0.6329652634000461 and parameters: {'encoding_dim': 17, 'learning_rate': 0.0002421844367805932, 'lr_C': 2.1241284026923077}. Best is trial 0 with value: 0.6329652634000461.
/Users/daklo/

Лучшие параметры:
    encoding_dim: 45
    learning_rate: 0.00017258037121608503
    lr_C: 0.0035482134223045894
Лучшее значение ROC-AUC: 0.6491


In [52]:
input_dim = X_train_.shape[1]
encoding_dim = 45
batch_size = 128
num_epochs = 65
learning_rate = 0.00017258037121608503

X_train_tensor = torch.FloatTensor(X_train_)
X_val_tensor = torch.FloatTensor(X_val_)

train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 186),
            nn.ReLU(),
            nn.Linear(186, encoding_dim),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 186),
            nn.ReLU(),
            nn.Linear(186, input_dim)
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

models = []
for i in range(1):
    model = Autoencoder(input_dim, encoding_dim)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch_inputs, batch_targets in train_loader:
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * batch_inputs.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_inputs, batch_targets in val_loader:
                outputs = model(batch_inputs)
                loss = criterion(outputs, batch_targets)
                val_loss += loss.item() * batch_inputs.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)
    models.append(model)

trains = []
vals = []
for model in models:
    encoder = model.encoder
    train_encoded = encoder(torch.FloatTensor(X_train_))
    trains.append(train_encoded)
    val_encoded = encoder(torch.FloatTensor(X_val_))
    vals.append(val_encoded)
train_encoded = torch.cat(trains, axis=1)
val_encoded = torch.cat(vals, axis=1)

with torch.no_grad():
    selector = SelectKBest(f_classif, k=75)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    ss = StandardScaler()
    with torch.no_grad():
        X_train = ss.fit_transform(X_train_s)
        X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_rr = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_rr:.4f}')

with torch.no_grad():
    selector = SelectKBest(f_classif, k=75)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
    X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)

    ss = StandardScaler()
    with torch.no_grad():
        X_train = ss.fit_transform(X_train_s)
        X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_main = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_main:.4f}')

ROC-AUC train autoencoded: 0.6512
ROC-AUC autoencoded: 0.6288
ROC-AUC train autoencoded: 0.7231
ROC-AUC autoencoded: 0.6318


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=75 is greater than n_features=45. All the features will be returned.
  warnings.warn(
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [13 27 38 40] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=75 is greater than n_features=45. All the features will be returned.
  warnings.warn(
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Featu

In [61]:
rocs = []
rocs_a = []
for _ in range(5):
    for i in range(1, 6):
        train_snp = pd.read_csv(f"./csv/train_{i}_snp_selected.csv")
        test_snp = pd.read_csv(f"./csv/test_{i}_snp_selected.csv")

        X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
        X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
        y_train = train_snp["target"] - 1
        y_val = test_snp["target"] - 1

        input_dim = X_train_.shape[1]
        encoding_dim = 45
        batch_size = 128
        num_epochs = 65
        learning_rate = 0.00017258037121608503

        X_train_tensor = torch.FloatTensor(X_train_)
        X_val_tensor = torch.FloatTensor(X_val_)

        train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        class Autoencoder(nn.Module):
            def __init__(self, input_dim, encoding_dim):
                super(Autoencoder, self).__init__()
                
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, 186),
                    nn.ReLU(),
                    nn.Linear(186, encoding_dim),
                    nn.ReLU()
                )
                
                self.decoder = nn.Sequential(
                    nn.Linear(encoding_dim, 186),
                    nn.ReLU(),
                    nn.Linear(186, input_dim)
                )
                
            def forward(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

        models = []
        for i in range(1):
            model = Autoencoder(input_dim, encoding_dim)

            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            train_losses = []
            val_losses = []

            for epoch in range(num_epochs):
                model.train()
                train_loss = 0.0
                
                for batch_inputs, batch_targets in train_loader:
                    outputs = model(batch_inputs)
                    loss = criterion(outputs, batch_targets)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item() * batch_inputs.size(0)
                
                train_loss = train_loss / len(train_loader.dataset)
                train_losses.append(train_loss)
                
                model.eval()
                val_loss = 0.0
                
                with torch.no_grad():
                    for batch_inputs, batch_targets in val_loader:
                        outputs = model(batch_inputs)
                        loss = criterion(outputs, batch_targets)
                        val_loss += loss.item() * batch_inputs.size(0)
                
                val_loss = val_loss / len(val_loader.dataset)
                val_losses.append(val_loss)
            models.append(model)

        trains = []
        vals = []
        for model in models:
            encoder = model.encoder
            train_encoded = encoder(torch.FloatTensor(X_train_))
            trains.append(train_encoded)
            val_encoded = encoder(torch.FloatTensor(X_val_))
            vals.append(val_encoded)
        train_encoded = torch.cat(trains, axis=1)
        val_encoded = torch.cat(vals, axis=1)

        with torch.no_grad():
            selector = SelectKBest(f_classif, k=75)
            X_train_s = selector.fit_transform(train_encoded, y_train)
            X_val_s = selector.transform(val_encoded)

            ss = StandardScaler()
            with torch.no_grad():
                X_train = ss.fit_transform(X_train_s)
                X_val = ss.transform(X_val_s)

            model_gb = LogisticRegression()
            model_gb.fit(X_train, y_train)

            y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
            y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

            roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
            print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
            roc_auc_rr = roc_auc_score(y_val, y_pred_proba)
            rocs_a.append(roc_auc_rr)
            print(f'ROC-AUC autoencoded: {roc_auc_rr:.4f}')

        with torch.no_grad():
            selector = SelectKBest(f_classif, k=75)
            X_train_s = selector.fit_transform(train_encoded, y_train)
            X_val_s = selector.transform(val_encoded)

            X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
            X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)

            ss = StandardScaler()
            with torch.no_grad():
                X_train = ss.fit_transform(X_train_s)
                X_val = ss.transform(X_val_s)

            model_gb = LogisticRegression()
            model_gb.fit(X_train, y_train)

            y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
            y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

            roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
            print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
            roc_auc_main = roc_auc_score(y_val, y_pred_proba)
            rocs.append(roc_auc_main)
            print(f'ROC-AUC autoencoded: {roc_auc_main:.4f}')
print(np.mean(rocs))
print(np.mean(roc_auc_rr))

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


ROC-AUC train autoencoded: 0.6818
ROC-AUC autoencoded: 0.5552
ROC-AUC train autoencoded: 0.7366
ROC-AUC autoencoded: 0.5586


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


ROC-AUC train autoencoded: 0.6820
ROC-AUC autoencoded: 0.5935
ROC-AUC train autoencoded: 0.7388
ROC-AUC autoencoded: 0.5712


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


...

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


ROC-AUC train autoencoded: 0.6723
ROC-AUC autoencoded: 0.5723
ROC-AUC train autoencoded: 0.7417
ROC-AUC autoencoded: 0.5792


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


ROC-AUC train autoencoded: 0.6562
ROC-AUC autoencoded: 0.5502
ROC-AUC train autoencoded: 0.7442
ROC-AUC autoencoded: 0.5153
ROC-AUC train autoencoded: 0.6696
ROC-AUC autoencoded: 0.6109
ROC-AUC train autoencoded: 0.7203
ROC-AUC autoencoded: 0.6294
0.5718193141560062
0.6109009009009009


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [90]:
input_dim = X_train_.shape[1]
encoding_dim = 45
batch_size = 128
num_epochs = 65
learning_rate = 0.00017258037121608503
classification_weight = 0.3

X_train_tensor = torch.FloatTensor(X_train_)
X_val_tensor = torch.FloatTensor(X_val_)
y_train_tensor = torch.FloatTensor(y_train)
y_val_tensor = torch.FloatTensor(y_val)

train_dataset = TensorDataset(X_train_tensor, X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

class AutoencoderWithClassifier(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoencoderWithClassifier, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 186),
            nn.ReLU(),
            nn.Linear(186, encoding_dim),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 186),
            nn.ReLU(),
            nn.Linear(186, input_dim)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)       
        decoded = self.decoder(encoded)
        classification = self.classifier(encoded)
        
        return decoded, classification.squeeze()

models = []
for i in range(1):
    model = AutoencoderWithClassifier(input_dim, encoding_dim)

    reconstruction_criterion = nn.MSELoss()
    classification_criterion = nn.BCELoss()
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch_inputs, batch_targets, batch_labels in train_loader:
            reconstructed, classification_output = model(batch_inputs)
            
            reconstruction_loss = reconstruction_criterion(reconstructed, batch_targets)
            classification_loss = classification_criterion(classification_output, batch_labels)
            
            total_loss = reconstruction_loss + classification_weight * classification_loss
            
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            
            train_loss += total_loss.item() * batch_inputs.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_inputs, batch_targets, batch_labels in val_loader:
                reconstructed, classification_output = model(batch_inputs)
                reconstruction_loss = reconstruction_criterion(reconstructed, batch_targets)
                classification_loss = classification_criterion(classification_output, batch_labels)
                total_loss = reconstruction_loss + classification_weight * classification_loss
                val_loss += total_loss.item() * batch_inputs.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
    
    models.append(model)

trains = []
vals = []
for model in models:
    encoder = model.encoder
    train_encoded = encoder(torch.FloatTensor(X_train_))
    trains.append(train_encoded)
    val_encoded = encoder(torch.FloatTensor(X_val_))
    vals.append(val_encoded)
train_encoded = torch.cat(trains, axis=1)
val_encoded = torch.cat(vals, axis=1)

with torch.no_grad():
    selector = SelectKBest(f_classif, k=75)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_s)
    X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_rr = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_rr:.4f}')

with torch.no_grad():
    selector = SelectKBest(f_classif, k=75)
    X_train_s = selector.fit_transform(train_encoded, y_train)
    X_val_s = selector.transform(val_encoded)

    X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
    X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_s)
    X_val = ss.transform(X_val_s)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
    y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

    roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
    print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
    roc_auc_main = roc_auc_score(y_val, y_pred_proba)
    print(f'ROC-AUC autoencoded: {roc_auc_main:.4f}')

with torch.no_grad():
    for model in models:
        _, train_predictions = model(torch.FloatTensor(X_train_))
        _, val_predictions = model(torch.FloatTensor(X_val_))
        
        classifier_train_auc = roc_auc_score(y_train, train_predictions.numpy())
        classifier_val_auc = roc_auc_score(y_val, val_predictions.numpy())
        
        print(f'Встроенный классификатор - ROC-AUC на обучающей выборке: {classifier_train_auc:.4f}')
        print(f'Встроенный классификатор - ROC-AUC на валидационной выборке: {classifier_val_auc:.4f}')


Epoch 10/65, Train Loss: 0.583368, Val Loss: 0.579047
Epoch 20/65, Train Loss: 0.555586, Val Loss: 0.558794
Epoch 30/65, Train Loss: 0.536546, Val Loss: 0.541883
Epoch 40/65, Train Loss: 0.509184, Val Loss: 0.514366
Epoch 50/65, Train Loss: 0.478382, Val Loss: 0.484500
Epoch 60/65, Train Loss: 0.447439, Val Loss: 0.455426
ROC-AUC train autoencoded: 0.6769
ROC-AUC autoencoded: 0.6301
ROC-AUC train autoencoded: 0.7288
ROC-AUC autoencoded: 0.5906
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.6511
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.6613


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide



In [93]:
rocs = []
rocs_ao = []
rocs_at = []
for j in range(7):
    for i in range(1, 6):
        train_snp = pd.read_csv(f"./csv/train_{i}_snp_selected.csv")
        test_snp = pd.read_csv(f"./csv/test_{i}_snp_selected.csv")

        X_train_ = train_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
        X_val_ = test_snp.drop(columns=["Unnamed: 0", "target"]).to_numpy()
        y_train = train_snp["target"] - 1
        y_val = test_snp["target"] - 1

        input_dim = X_train_.shape[1]
        encoding_dim = 45
        batch_size = 128
        num_epochs = 65
        learning_rate = 0.00017258037121608503
        classification_weight = 0.3

        X_train_tensor = torch.FloatTensor(X_train_)
        X_val_tensor = torch.FloatTensor(X_val_)
        y_train_tensor = torch.FloatTensor(y_train)
        y_val_tensor = torch.FloatTensor(y_val)

        train_dataset = TensorDataset(X_train_tensor, X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, X_val_tensor, y_val_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        class AutoencoderWithClassifier(nn.Module):
            def __init__(self, input_dim, encoding_dim):
                super(AutoencoderWithClassifier, self).__init__()
                
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, 186),
                    nn.ReLU(),
                    nn.Linear(186, encoding_dim),
                    nn.ReLU()
                )
                
                self.decoder = nn.Sequential(
                    nn.Linear(encoding_dim, 186),
                    nn.ReLU(),
                    nn.Linear(186, input_dim)
                )
                
                self.classifier = nn.Sequential(
                    nn.Linear(encoding_dim, 64),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(64, 32),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(32, 1),
                    nn.Sigmoid()
                )
                
            def forward(self, x):
                encoded = self.encoder(x)       
                decoded = self.decoder(encoded)
                classification = self.classifier(encoded)
                
                return decoded, classification.squeeze()

        models = []
        for i in range(1):
            model = AutoencoderWithClassifier(input_dim, encoding_dim)

            reconstruction_criterion = nn.MSELoss()
            classification_criterion = nn.BCELoss()
            
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            train_losses = []
            val_losses = []

            for epoch in range(num_epochs):
                model.train()
                train_loss = 0.0
                
                for batch_inputs, batch_targets, batch_labels in train_loader:
                    reconstructed, classification_output = model(batch_inputs)
                    
                    reconstruction_loss = reconstruction_criterion(reconstructed, batch_targets)
                    classification_loss = classification_criterion(classification_output, batch_labels)
                    
                    total_loss = reconstruction_loss + classification_weight * classification_loss
                    
                    optimizer.zero_grad()
                    total_loss.backward()
                    optimizer.step()
                    
                    train_loss += total_loss.item() * batch_inputs.size(0)
                
                train_loss = train_loss / len(train_loader.dataset)
                train_losses.append(train_loss)
                
                model.eval()
                val_loss = 0.0
                
                with torch.no_grad():
                    for batch_inputs, batch_targets, batch_labels in val_loader:
                        reconstructed, classification_output = model(batch_inputs)
                        reconstruction_loss = reconstruction_criterion(reconstructed, batch_targets)
                        classification_loss = classification_criterion(classification_output, batch_labels)
                        total_loss = reconstruction_loss + classification_weight * classification_loss
                        val_loss += total_loss.item() * batch_inputs.size(0)
                
                val_loss = val_loss / len(val_loader.dataset)
                val_losses.append(val_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
            
            models.append(model)

        trains = []
        vals = []
        for model in models:
            encoder = model.encoder
            train_encoded = encoder(torch.FloatTensor(X_train_))
            trains.append(train_encoded)
            val_encoded = encoder(torch.FloatTensor(X_val_))
            vals.append(val_encoded)
        train_encoded = torch.cat(trains, axis=1)
        val_encoded = torch.cat(vals, axis=1)

        with torch.no_grad():
            selector = SelectKBest(f_classif, k=75)
            X_train_s = selector.fit_transform(train_encoded, y_train)
            X_val_s = selector.transform(val_encoded)

            ss = StandardScaler()
            X_train = ss.fit_transform(X_train_s)
            X_val = ss.transform(X_val_s)

            model_gb = LogisticRegression()
            model_gb.fit(X_train, y_train)

            y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
            y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

            roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
            print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
            roc_auc_rr = roc_auc_score(y_val, y_pred_proba)
            rocs_ao.append(roc_auc_rr)
            print(f'ROC-AUC autoencoded: {roc_auc_rr:.4f}')

        with torch.no_grad():
            selector = SelectKBest(f_classif, k=75)
            X_train_s = selector.fit_transform(train_encoded, y_train)
            X_val_s = selector.transform(val_encoded)

            X_train_s = torch.cat([torch.FloatTensor(X_train_s), torch.FloatTensor(X_train_)], axis=1)
            X_val_s = torch.cat([torch.FloatTensor(X_val_s), torch.FloatTensor(X_val_)], axis=1)

            ss = StandardScaler()
            X_train = ss.fit_transform(X_train_s)
            X_val = ss.transform(X_val_s)

            model_gb = LogisticRegression()
            model_gb.fit(X_train, y_train)

            y_pred_proba = model_gb.predict_proba(X_val)[:, 1]
            y_pred_proba_tra = model_gb.predict_proba(X_train)[:, 1]

            roc_auc = roc_auc_score(y_train, y_pred_proba_tra)
            print(f'ROC-AUC train autoencoded: {roc_auc:.4f}')
            roc_auc_main = roc_auc_score(y_val, y_pred_proba)
            rocs_at.append(roc_auc_main)
            print(f'ROC-AUC autoencoded: {roc_auc_main:.4f}')

        with torch.no_grad():
            for model in models:
                _, train_predictions = model(torch.FloatTensor(X_train_))
                _, val_predictions = model(torch.FloatTensor(X_val_))
                
                classifier_train_auc = roc_auc_score(y_train, train_predictions.numpy())
                classifier_val_auc = roc_auc_score(y_val, val_predictions.numpy())
                rocs.append(classifier_val_auc)
                
                print(f'Встроенный классификатор - ROC-AUC на обучающей выборке: {classifier_train_auc:.4f}')
                print(f'Встроенный классификатор - ROC-AUC на валидационной выборке: {classifier_val_auc:.4f}')
print(np.mean(rocs), "средний ROC-AUC для классифицирующей головы")
print(np.mean(rocs_ao), "средний ROC-AUC для логистической регрессии на латентном пространстве")
print(np.mean(rocs_at), "средний ROC-AUC для логистической регрессии на латентном пространстве и изначальных признаках")

Epoch 10/65, Train Loss: 0.570884, Val Loss: 0.568512
Epoch 20/65, Train Loss: 0.549860, Val Loss: 0.554198
Epoch 30/65, Train Loss: 0.527036, Val Loss: 0.533605
Epoch 40/65, Train Loss: 0.489017, Val Loss: 0.502101
Epoch 50/65, Train Loss: 0.453225, Val Loss: 0.473089
Epoch 60/65, Train Loss: 0.420354, Val Loss: 0.447240
ROC-AUC train autoencoded: 0.7233
ROC-AUC autoencoded: 0.5274


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide



ROC-AUC train autoencoded: 0.7795
ROC-AUC autoencoded: 0.5817
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.7045
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.5489
Epoch 10/65, Train Loss: 0.593641, Val Loss: 0.580345
Epoch 20/65, Train Loss: 0.555946, Val Loss: 0.556288
Epoch 30/65, Train Loss: 0.542124, Val Loss: 0.543851
Epoch 40/65, Train Loss: 0.511716, Val Loss: 0.519895
Epoch 50/65, Train Loss: 0.476633, Val Loss: 0.493833
Epoch 60/65, Train Loss: 0.443547, Val Loss: 0.469628
ROC-AUC train autoencoded: 0.7310
ROC-AUC autoencoded: 0.5794


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide



ROC-AUC train autoencoded: 0.8226
ROC-AUC autoencoded: 0.5812
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.7040
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.5586
Epoch 10/65, Train Loss: 0.582223, Val Loss: 0.571351
Epoch 20/65, Train Loss: 0.555651, Val Loss: 0.551865
Epoch 30/65, Train Loss: 0.542214, Val Loss: 0.537918
Epoch 40/65, Train Loss: 0.508889, Val Loss: 0.512040
Epoch 50/65, Train Loss: 0.467897, Val Loss: 0.482141
Epoch 60/65, Train Loss: 0.431367, Val Loss: 0.457932


...

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide



ROC-AUC train autoencoded: 0.7828
ROC-AUC autoencoded: 0.5940
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.7002
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.6107
Epoch 10/65, Train Loss: 0.581527, Val Loss: 0.575858
Epoch 20/65, Train Loss: 0.551538, Val Loss: 0.555924
Epoch 30/65, Train Loss: 0.524277, Val Loss: 0.537657
Epoch 40/65, Train Loss: 0.484719, Val Loss: 0.510756
Epoch 50/65, Train Loss: 0.447574, Val Loss: 0.482561
Epoch 60/65, Train Loss: 0.412335, Val Loss: 0.457276
ROC-AUC train autoencoded: 0.7161
ROC-AUC autoencoded: 0.5535


/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide



ROC-AUC train autoencoded: 0.7655
ROC-AUC autoencoded: 0.5126
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.7032
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.5306
Epoch 10/65, Train Loss: 0.566790, Val Loss: 0.567798
Epoch 20/65, Train Loss: 0.549833, Val Loss: 0.552668
Epoch 30/65, Train Loss: 0.523226, Val Loss: 0.525408
Epoch 40/65, Train Loss: 0.480944, Val Loss: 0.484016
Epoch 50/65, Train Loss: 0.440117, Val Loss: 0.444620
Epoch 60/65, Train Loss: 0.403421, Val Loss: 0.414984
ROC-AUC train autoencoded: 0.7007
ROC-AUC autoencoded: 0.6325
ROC-AUC train autoencoded: 0.7608
ROC-AUC autoencoded: 0.6358
Встроенный классификатор - ROC-AUC на обучающей выборке: 0.6853
Встроенный классификатор - ROC-AUC на валидационной выборке: 0.6597
0.5803934053693927 средний ROC-AUC для классифицирующей головы
0.575060054904822 средний ROC-AUC для логистической регрессии на латентном пространстве
0.5683616623414982 средний ROC-AUC для логистической регрессии на латентном

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

/Users/daklo/for_dipl/for_dipl/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

